# PC Yearly Automatization

## Imports and Functions

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import scipy as sp
import os
from itertools import combinations

import sys
import seaborn as sns
import re
import pickle


import cartopy.crs as ccrs
import cartopy.feature as cfeature



In [2]:
# Add the folder to the Python path

os.chdir("../")
# change working directory to project's root path
print(os.getcwd())

folder_path = os.path.abspath("functions/") #INPUT_PATH)#'path_to_your_folder')  # Replace with the actual folder path
sys.path.insert(0, folder_path)

c:\Users\marti\Desktop\data\hw_extra


In [3]:
from PredictorsDrivers import (
    PCAPredictors
)

In [4]:
ds_6means = xr.load_dataset("data/local_data/6means_world.nc")
num_modes = 3
generate_pcas = False

In [5]:
if generate_pcas:
    predictors = PCAPredictors(ds_6means, num_modes, frequency="yearly")
else:
    with open("pcas_1972_yearly.pkl", "rb") as inp:
        pcas = pickle.load(inp)
    predictors = PCAPredictors(ds_6means, num_modes, frequency="yearly", saved_pcas=pcas)

c:\Users\marti\Desktop\data\hw_extra\functions\PredictorsDrivers.py:292: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"PC_{num}-Mode-{mode+1}"] = pca.get_index(mode+1, start_year=self.start_year, end_year=self.end_year)
c:\Users\marti\Desktop\data\hw_extra\functions\PredictorsDrivers.py:292: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"PC_{num}-Mode-{mode+1}"] = pca.get_index(mode+1, start_year=self.start_year, end_year=self.end_year)
c:\Users\marti\Desktop\data\hw_extra\functions\PredictorsDrivers.py:292: Performance

In [6]:
print(f"Total of different PCAS {len(predictors.df_predictors.columns)}")

Total of different PCAS 4536


## Chile

In [7]:
hwis_chile= pd.read_csv(f"data/local_data/yearly/HWs_chile_central.csv")
hwis_chile["Date"] = pd.to_datetime(hwis_chile["Date"],format='%Y')
hwis_chile.set_index('Date', inplace=True)
first_year = 1972
hwis_chile = hwis_chile[(hwis_chile.index.year <= 2022) & (hwis_chile.index.year >= first_year)]


In [8]:
top10, chile_experiment = predictors.top_correlations_predictors(hwis_chile)

In [9]:
#predictors.experiment_to_csv(chile_experiment,"data/features/chile/from1972/top10/","HWs_chile_features_yearly.csv")

### Save top but filtering low variance

In [10]:
top10_clean, chile_experiment_2 = predictors.top_correlations_predictors(hwis_chile, threshold_variance=0.15)

In [11]:
#predictors.experiment_to_csv(chile_experiment_2,"data/features/chile/from1972/top10/","HWs_chile_features_yearly.csv")

## California

In [12]:
hwis_cali= pd.read_csv(f"data/local_data/yearly/HWs_california_NOAA.csv")
hwis_cali["Date"] = pd.to_datetime(hwis_cali["Date"],format='%Y')
hwis_cali.set_index('Date', inplace=True)
hwis_cali = hwis_cali[(hwis_cali.index.year <= 2022) & (hwis_cali.index.year >= first_year)]

In [13]:
top10, cali_experiment = predictors.top_correlations_predictors(hwis_cali)

In [14]:
predictors.experiment_to_csv(cali_experiment,"data/features/california/from1972/top10/","HWs_cali_features_yearly.csv")

Saved


### Save top but filtering low variance

In [15]:
top10_clean, cali_experiment_2 = predictors.top_correlations_predictors(hwis_cali, threshold_variance=0.15)

In [16]:
predictors.experiment_to_csv(cali_experiment_2,"data/features/california/from1972/top10/","HWs_cali_features_yearly.csv")

Saved
